In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [20]:
df = pd.read_csv('/content/source_pour_score_eco_sportif_region.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   region                             264 non-null    object
 1   fin_saison                         264 non-null    object
 2   SUM de nb_clubs                    264 non-null    int64 
 3   SUM de nb_sport                    264 non-null    int64 
 4   AVERAGE de score_classement_moyen  264 non-null    int64 
 5   SUM de nb_crea_entreprise          264 non-null    object
 6   AVERAGE de taux_chomage            264 non-null    object
 7   SUM de salaire_median              264 non-null    object
dtypes: int64(3), object(5)
memory usage: 16.6+ KB


In [4]:
df['fin_saison'] = df['fin_saison'].str.replace(r'[^\d]', '', regex=True)

In [5]:
df['fin_saison'] = df['fin_saison'].astype(int)

In [6]:
df['SUM de salaire_median'] = df['SUM de salaire_median'].str.replace(r'[^\d]', '', regex=True)

In [7]:
df['SUM de salaire_median'] = df['SUM de salaire_median'].astype(int)

In [8]:
df['SUM de nb_crea_entreprise'] = df['SUM de nb_crea_entreprise'].str.replace(r'[^\d]', '', regex=True)

In [9]:
df['SUM de nb_crea_entreprise'] = df['SUM de nb_crea_entreprise'].astype(int)

In [10]:
df['AVERAGE de taux_chomage'] = df['AVERAGE de taux_chomage'].str.replace(',', '.')

df['AVERAGE de taux_chomage'] = pd.to_numeric(df['AVERAGE de taux_chomage'], errors='coerce')

In [11]:
df.rename(columns= {'fin_saison': 'annee', 'SUM de nb_clubs': 'nb_clubs', 'SUM de nb_sport': 'nb_sports', 'AVERAGE de score_classement_moyen': 'score_classement_moyen', 'SUM de nb_crea_entreprise': 'nb_crea_entreprise', 'AVERAGE de taux_chomage': 'taux_chomage', 'SUM de salaire_median': 'salaire_median'}, inplace= True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   region                  264 non-null    object 
 1   annee                   264 non-null    int64  
 2   nb_clubs                264 non-null    int64  
 3   nb_sports               264 non-null    int64  
 4   score_classement_moyen  264 non-null    int64  
 5   nb_crea_entreprise      264 non-null    int64  
 6   taux_chomage            264 non-null    float64
 7   salaire_median          264 non-null    int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 16.6+ KB


Création Score Sportif

In [ ]:
# coeff_licencies_foot_annee = licencies_foot / (licencies_foot + licenciés_rugby + licenciés_basket)

In [ ]:
# coeff_licencies_rugby_annee = licencies_rugby / (licencies_foot + licenciés_rugby + licenciés_basket)

In [ ]:
# coeff_licencies_basket_annee = licencies_basket / (licencies_foot + licenciés_rugby + licenciés_basket)

In [ ]:
# score_presence_pro_foot_annee = si club present en D1 alors +2, si club present en D2 alors +1, sinon 0 (A NORMALISER)

In [ ]:
# score_presence_pro_rugby_annee = si club present en D1 alors +2, si club present en D2 alors +1, sinon 0 (A NORMALISER)

In [ ]:
# score_presence_pro_basket_annee = si club present en D1 alors +2, si club present en D2 alors +1, sinon 0 (A NORMALISER)

In [ ]:
# score_popularite_foot = (coeff_licencies_foot_annee + score_presence_pro_foot_annee)

In [ ]:
# score_popularite_rugby = (coeff_licencies_rugby_annee + score_presence_pro_rugby_annee)

In [ ]:
# score_popularite_basket = (coeff_licencies_basket_annee + score_presence_pro_basket_annee)

In [ ]:
# chaque annee ville sport on a le score de popularite sur 1

In [ ]:
# par ville calculer la "présence sportive pour chaque sport, avec la part du nb de licenciés et la présence des clubs dans les div1 et 2"

In [ ]:
#j'ai les scores par sport

In [ ]:
# score_sportif = (score_foot * score_popularite_foot) + (score_rugby * score_popularite_rugby) + (score_basket * score_popularite_basket) / 3

In [13]:
df['score_sportif_norm'] = (df['score_classement_moyen'] - df['score_classement_moyen'].min()) / (df['score_classement_moyen'].max() - df['score_classement_moyen'].min())

Création Score économique

In [14]:
df['salaire_median_norm'] = (df['salaire_median'] - df['salaire_median'].min()) / (df['salaire_median'].max() - df['salaire_median'].min())

In [15]:
df['taux_chomage_norm'] = 1 - (df['taux_chomage'] - df['taux_chomage'].min()) / (df['taux_chomage'].max() - df['taux_chomage'].min())

In [16]:
df['nb_crea_entreprise_norm'] = (df['nb_crea_entreprise'] - df['nb_crea_entreprise'].min()) / (df['nb_crea_entreprise'].max() - df['nb_crea_entreprise'].min())

In [17]:
df['score_economique'] = (0.4 * df['salaire_median_norm']) + (0.5 * df['taux_chomage_norm']) + (0.5 * df['nb_crea_entreprise_norm'])

In [18]:
df['score_economique_norm'] = (df['score_economique'] - df['score_economique'].min()) / (df['score_economique'].max() - df['score_economique'].min())

Visualisation

In [19]:
# Pivot des données pour les regrouper par région et année
df_grouped = df.groupby(['region', 'annee']).agg({
    'score_sportif_norm': 'mean',
    'score_economique': 'mean'
}).reset_index()

# Création de la figure avec des traces pour chaque région
fig = go.Figure()

regions = df_grouped['region'].unique()

# Ajouter une trace pour chaque région
for region in regions:
    region_data = df_grouped[df_grouped['region'] == region]

    # Score Sportif
    fig.add_trace(go.Scatter(
        x=region_data['annee'],
        y=region_data['score_sportif_norm'],
        mode='lines+markers',
        name=f"Sportif ({region})",
        line=dict(dash="solid"),
        visible=False  # Initialement masqué
    ))

    # Score Économique
    fig.add_trace(go.Scatter(
        x=region_data['annee'],
        y=region_data['score_economique'],
        mode='lines+markers',
        name=f"Économique ({region})",
        line=dict(dash="dot"),
        visible=False  # Initialement masqué
    ))

# Activer la première région par défaut
fig.data[0].visible = True
fig.data[1].visible = True

# Boutons interactifs pour sélectionner la région
buttons = []
for i, region in enumerate(regions):
    # Index des traces pour la région
    sportif_idx = 2 * i
    economique_idx = 2 * i + 1

    # Création du bouton
    buttons.append(dict(
        label=region,
        method="update",
        args=[{"visible": [j == sportif_idx or j == economique_idx for j in range(len(fig.data))]},
              {"title": f"Évolution des Scores pour {region}"}]
    ))

# Ajout du menu dropdown
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.15,
        xanchor="right",
        y=1,
        yanchor="top"
    )],
    title="Évolution des Scores Sportif et Économique",
    xaxis_title="Année",
    yaxis_title="Score",
    legend_title="Type de Score",
    template="plotly_white"
)

# Afficher le graphique
fig.show()

In [ ]:
# Calcul de la corrélation par région
correlation_by_region = df.groupby('region').apply(
    lambda group: group['score_sportif_norm'].corr(group['score_economique_norm'])
)

# Convertir en DataFrame pour une visualisation plus simple
correlation_by_region = correlation_by_region.reset_index(name='correlation')

# Trier les régions par corrélation décroissante
correlation_by_region = correlation_by_region.sort_values(by='correlation', ascending=False)

# Afficher les résultats
correlation_by_region

<ipython-input-105-2b5b1c02440e>:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,region,correlation
3,Basse-Normandie,0.826063
2,Auvergne,0.552591
18,Poitou-Charentes,0.507058
7,Champagne-Ardenne,0.460964
16,Pays de la Loire,0.379258
12,Limousin,0.346592
5,Bretagne,0.344032
10,Haute-Normandie,0.308918
15,Nord-Pas-de-Calais,0.258272
6,Centre,0.245170
